In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from IPython.display import FileLink

import base64
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import pyLDAvis.sklearn
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)



In [2]:
import gensim
all_stopwords = gensim.parsing.preprocessing.STOPWORDS

In [13]:
import urllib3

http = urllib3.PoolManager()
response = http.request('GET', "https://raw.githubusercontent.com/chaskaraditi/Takeaways/master/Takeaways.txt")
document = response.data.decode('utf-8')


/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
list_of_sentence=document.split("\n")

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
list_of_simple_preprocess_data = []
for i in list_of_sentence:
    list_of_simple_preprocess_data.append(gensim.utils.simple_preprocess(i, deacc=True, min_len=2))

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
data = list_of_simple_preprocess_data

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data[0]]])

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in all_stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))

/home/chaskaraditi/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Visualization  
  
Steps:-    
- From the "Number" dropdown select the number of topics to visualize  
- Multidimensional scaling and top relevant terms visualization would be changed depending upon the selected number of topics  
- "Dominant" dropdown displays the list of dominant topics  
- Table below the Dominant dropdown reflects the "Keywords", "Percentage" and "Text" of the selected dominant topic  


In [1]:
from ipywidgets import interact, fixed
from IPython.core.interactiveshell import InteractiveShell
import ipywidgets as widgets
from IPython.display import display
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

a_list = list(range(2, 51))
@interact(Number=a_list)
def my_function(Number):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=Number, id2word=id2word,random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    pyLDAvis.enable_notebook()
    p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word,R=10, sort_topics=False, mds='mmds')
    display(p)
    def format_topics_sentences(ldamodel=None, corpus=corpus, texts=list_of_sentence):
    # Init output
        sent_topics_df = pd.DataFrame()
        pd.set_option('max_colwidth',None)


        # Get main topic in each document
        for i, row_list in enumerate(ldamodel[corpus]):
            row = row_list[0] if ldamodel.per_word_topics else row_list            
            # print(row)
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if ((j == 0) & (prop_topic >0.6)):  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

        # Add original text to the end of the output
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
        return(sent_topics_df)


    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=list_of_sentence)

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Percentage', 'Keywords', 'Text']
    df_dominant_topic['Dominant_Topic'] = df_dominant_topic['Dominant_Topic'] + 1

    #ALL = 'ALL'
    def unique_sorted_values_plus_ALL(array):
        colvalue = array.unique().tolist()
        colvalue.sort()
        #colvalue.insert(0, ALL)
        return colvalue

    this =unique_sorted_values_plus_ALL(df_dominant_topic['Dominant_Topic'].value_counts().loc[lambda x : x>3].reset_index()['index'])
    @interact(DominantTopic=this)
    def my_function(DominantTopic):
        df_dominant = (df_dominant_topic[df_dominant_topic['Dominant_Topic'] == DominantTopic]).sort_values(by=['Percentage'],ascending=False)
        download = (df_dominant[['Keywords', 'Percentage','Text']])
        StyleDownload = download.style.hide_index()
        displayDF = StyleDownload.set_properties(**{'text-align': 'left'}).set_properties(subset=['Keywords'], **{'width': '300px'})
        display(displayDF.set_table_styles([dict(selector='th', props=[('text-align', 'left')])]))
        download.to_csv("TopicModel-%i-%i-document.csv" %(Number,DominantTopic))
        download.to_csv("output.csv",index=False)
        
    def edit_download_html(htmlWidget, filename, title = "Click here to download: "):

    # Change widget html temperarily to a font-awesome spinner
        htmlWidget.value = "<i class=\"fa fa-spinner fa-spin fa-2x fa-fw\"></i><span class=\"sr-only\">Loading...</span>"

        # Process raw data
        data = open(filename, "rb").read()
        b64 = base64.b64encode(data)
        payload = b64.decode()

        # Create and assign html to widget
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        htmlWidget.value = html.format(payload = payload, title = title+filename, filename = filename)

    htmlWidget = widgets.HTML(value = '')
    display(htmlWidget)
    filename = r"output.csv"
    title = "Click here to download: "
    edit_download_html(htmlWidget, filename, title = title)
    

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


<IPython.core.display.Javascript object>

NameError: name 'pd' is not defined